In [1]:
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install fastf1

Note: you may need to restart the kernel to use updated packages.


In [7]:
import fastf1
import numpy as np
import seaborn as sns
import pandas as pd

#--
import fastf1

fastf1.Cache.enable_cache('f1_cache')

#2
session = fastf1.get_session(2023, 'Monza', 'R')
session.load()

core           INFO 	Loading data for Italian Grand Prix - Race [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

#Preparation and Logic

In [11]:
laps = session.laps
laps.head()
laps.columns


Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime',
       'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
       'FastF1Generated', 'IsAccurate'],
      dtype='object')

In [14]:
print(laps.head())

                    Time Driver DriverNumber                LapTime  \
0 0 days 01:23:49.780000    VER            1 0 days 00:01:27.905000   
1 0 days 01:25:15.943000    VER            1 0 days 00:01:26.163000   
2 0 days 01:26:41.645000    VER            1 0 days 00:01:25.702000   
3 0 days 01:28:07.695000    VER            1 0 days 00:01:26.050000   
4 0 days 01:29:33.871000    VER            1 0 days 00:01:26.176000   

   LapNumber  Stint PitOutTime PitInTime            Sector1Time  \
0        1.0    1.0        NaT       NaT                    NaT   
1        2.0    1.0        NaT       NaT 0 days 00:00:28.274000   
2        3.0    1.0        NaT       NaT 0 days 00:00:28.097000   
3        4.0    1.0        NaT       NaT 0 days 00:00:28.145000   
4        5.0    1.0        NaT       NaT 0 days 00:00:28.155000   

             Sector2Time  ...             Team           LapStartTime  \
0 0 days 00:00:29.666000  ...  Red Bull Racing 0 days 01:22:21.726000   
1 0 days 00:00:29.543000

In [10]:
print(laps[['Driver', 'LapNumber', 'LapTime', 'TyreLife', 'Compound']].head(40))

   Driver  LapNumber                LapTime  TyreLife Compound
0     VER        1.0 0 days 00:01:27.905000       1.0   MEDIUM
1     VER        2.0 0 days 00:01:26.163000       2.0   MEDIUM
2     VER        3.0 0 days 00:01:25.702000       3.0   MEDIUM
3     VER        4.0 0 days 00:01:26.050000       4.0   MEDIUM
4     VER        5.0 0 days 00:01:26.176000       5.0   MEDIUM
5     VER        6.0 0 days 00:01:26.837000       6.0   MEDIUM
6     VER        7.0 0 days 00:01:26.368000       7.0   MEDIUM
7     VER        8.0 0 days 00:01:25.990000       8.0   MEDIUM
8     VER        9.0 0 days 00:01:26.605000       9.0   MEDIUM
9     VER       10.0 0 days 00:01:26.603000      10.0   MEDIUM
10    VER       11.0 0 days 00:01:26.643000      11.0   MEDIUM
11    VER       12.0 0 days 00:01:26.442000      12.0   MEDIUM
12    VER       13.0 0 days 00:01:26.666000      13.0   MEDIUM
13    VER       14.0 0 days 00:01:27.028000      14.0   MEDIUM
14    VER       15.0 0 days 00:01:26.782000      15.0  

In [ ]:
#obtain the laps
laps['LapTimeSeconds'] = laps['LapTime'].dt.total_seconds()


In [13]:
'''
Calculating Pitloss
Here we compare green flag lap time vs the Pit in and out lap times
'''
# Filter for drivers who finished, doing this removes crash data (drivers who crashed during the race)
clean_laps = laps.pick_quicklaps() # Removes slow laps ( like Yellow flags)
avg_race_pace = clean_laps['LapTimeSeconds'].median()

In [16]:
#Identify Pit Laps
pit_laps = laps[laps['PitOutTime'].notna()] # Laps where they exited the pits
avg_pit_lap_time = pit_laps['LapTimeSeconds'].median()

In [17]:
# Estimating PIt loss
pit_loss = avg_pit_lap_time - avg_race_pace

print(f"Avg Race Pace: {avg_race_pace} seconds")

print(f"Avg Pit lap Time: {avg_pit_lap_time} seconds")

print(f"Estimated Pit loss: {pit_loss} seconds")

Avg Race Pace: 86.968 seconds
Avg Pit lap Time: 107.237 seconds
Estimated Pit loss: 20.26899999999999 seconds


# Modeling Tire Degradation (how much slower a car gets per lap)

In [ ]:
#Making a tire degradation function and a rival bot, Later, the goal is to beat this bot.

import random

class RaceParameters:
    base_lap_time = 87
    pit_loss = 21.00
def get_tire_degradation(tire_age, compound='medium'):
    if compound == 'soft':
        deg = 0.15 * tire_age
        if tire_age > 15: deg += 0.5 * (tire_age - 15)
    elif compound == 'medium':
        deg = 0.08 * tire_age
        if tire_age > 25: deg += 0.4 * (tire_age - 25)
    elif compound == 'hard':
        deg = 0.05 * tire_age
        if tire_age > 35: deg += 0.3 * (tire_age - 35)
    else:
        raise ValueError(f"Unknown compound: {compound}")
    return deg

class RivalBot:
    def __init__(self, pit_lap=24, start_tire='medium', switch_tire='hard'):
        self.pit_lap = pit_lap
        self.current_tire = start_tire
        self.switch_tire = switch_tire
        self.tire_age = 0

    def get_action(self, current_lap):
        if current_lap == self.pit_lap:
            self.current_tire = self.switch_tire
            self.tire_age = 0
            return 1  # Pit
        
        self.tire_age += 1
        return 0  # Stay Out


#Design